### 第六章 数据加载、存储与文件格式

#####    6.1 读写文本格式的数据
    pandas提供的用于将表格型数据读取为DataFrame对象的函数。（read_csv和read_table会是用的比较多的）

    read_csv----有超过50个参数
    read_table
    read_fwf（读取定宽列格式数据，也就是说没有分隔符）
    read_clipboard（读取剪切板中的数据，网页转换为表格时很常用）
    read_excel
    read_hdf（读取pandas写的HDF5文件）
    read_json
    read_html（读取HTML文档中的所有表格）
    read_msgpack（读取二进制格式编码的pandas数据）
    read_pickle（读取python pickle格式中存储的任意对象）
    read_sas（读取存储于SAS系统自定义存储格式的SAS数据集）
    read_sql（读取sql查询结果为pandas的DataFrame）
    read_stata（读取stata文件格式的数据集）
    read_feather（读取feather二进制文件）

In [1]:
import numpy as np
import pandas as pd

In [9]:
# !cat examples/ex1.csv
# # 这是Linux下的用法
!type examples\ex1.csv
# # 这是windows下的用法

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
df = pd.read_csv('examples/ex1.csv')

In [3]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# 采用 read_table 函数，指定分隔符（该函数原分隔符是制表符）
pd.read_table('examples/ex1.csv', sep=',')

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    并不是所有文件都有标题行，读入没有标题行的文件，有两个方法，可以让pandas为其分配默认的列名，也可以自己定义列名

In [9]:
# pd.read_csv('examples/ex2.csv', header=None)
pd.read_csv('examples/ex2.csv')
# 没有设置header参数，默认将第一行当做了标题行

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [5]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 自己设置索引，可以明确表示要将该列放到索引4的位置，也可以通过index_col参数指定‘message’
names=['a', 'b', 'c', 'd', 'message']

In [8]:
pd.read_csv('examples/ex2.csv', names=names, index_col='message')
# 设置message列的数据作为索引

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


    传入由列编号或列名组成的列表，做一个层次化的索引

In [11]:
parsed = pd.read_csv('examples/csv_mindex.csv')
parsed

,key1,key2,value1,value2
0,one,a,1,2
1,one,b,3,4
2,one,c,5,6
3,one,d,7,8
4,two,a,9,10
5,two,b,11,12
6,two,c,13,14
7,two,d,15,16


In [9]:
parsed = pd.read_csv('examples/csv_mindex.csv', index_col=['key1', 'key2'])

In [10]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
# 有的表格可能不是用固定的分隔符取分隔字段

In [13]:
list(open('examples/ex3.txt'))
# 文件里面的字段是被数量不同的空白字符间隔开的

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [14]:
#  可以传递一个正则表达式作为read_table的分隔符。
result = pd.read_table('examples/ex3.txt', sep='\s+')
# 正则表达式：\s+ 匹配多个字符

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  


In [15]:
result
# 由于列名比数据行的数量少，所以read_table推断第一列应该是DataFrame的索引。

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [17]:
pd.read_csv('examples/ex4.csv')

,,,,# hey!
a,b,c,d,message
# just wanted to make things more difficult for you,NaN,NaN,NaN,NaN
# who reads CSV files with computers,anyway?,NaN,NaN,NaN
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
#  skiprows参数，设置读取时跳过某行
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])
# 跳过了 0 2 3 行

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    缺失值的处理：缺失数据经常是要么没有（空字符串），要么用某个标记值表示。默认情况下，pandas会用一组经常出现的标记值进行识别，比如NA及NULL:

In [18]:
result = pd.read_csv('examples/ex5.csv')

In [19]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [20]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


    na_values可以用一个列表或集合的字符串表示缺失值：

In [21]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])

In [22]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


    字典中的各列可以用不同的NA标记值

In [31]:
sentinels = {
    'message': ['foo', 'NA'],
    'something': ['two']
}
# message 的foo，something的two，处理为缺失值

In [32]:
pd.read_csv('examples/ex5.csv', na_values=sentinels)
# pd.read_csv('examples/ex5.csv', na_values=None)
# na_values 一组用于替换NA的值,定义另外的应处理为缺失值的值

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


##### 6.1.1 逐块读取文本文件

In [2]:
# 先设置pandas显示
pd.options.display.max_rows = 10
# 这里设置l

In [3]:
result = pd.read_csv('examples/ex6.csv')

In [4]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


    如果只想读取几行，（避免读取整个文件）通过nrow进行制定即可

In [5]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


    要逐块读取文件，可以指定chunksize（行数）：

In [6]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)

In [8]:
chunker
# read_csv所返回的TextParser对象使你可以根据chunksize对文件进行逐块迭代。

In [9]:
# 比如：迭代处理ex6.csv，将值计数聚合到key列中
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

# 解读一下：
#     ex6.csv文件中的列：one  two  three four  key
#     设置了行数为1000的代码块
#     定义了一个Series，初始值为空
#     Series的add方法，进行加法计算
#     value_counts，计算数据的频数，计算了每一个key列数据出现的次数
#     fill_value，确实值填充为0

# TextParser还有一个get_chunk方法，可以读取任意大小的块

In [19]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

In [24]:
chunksize = 1000
result = pd.read_csv('examples/ex6.csv', iterator=True)
chunks = []
while True:
    try:
        chunk = result.get_chunk(chunksize) #一次获得1kw的数据量
        chunks.append(chunk)
    except StopIteration:
        print("Iteration is stopped.")
        break

# 以上使用了get_chunk方法来将大量的数据进行分块处理。
# 下面的处理和上一部分一样，将值计数聚合到key列

tot = pd.Series([])
for piece in chunks:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

Iteration is stopped.


##### 6.1.2 将数据写出到文本格式

In [26]:
data = pd.read_csv('examples/ex5.csv')

In [27]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


    利用DataFrame的to_csv方法，我们可以将数据写到一个以逗号分隔的文件中：

In [28]:
data.to_csv('examples/out.csv')

In [29]:
pd.read_csv('examples/out.csv')

,Unnamed: 0,something,a,b,c,d,message
0,0,one,1,2,3.0,4,NaN
1,1,two,5,6,NaN,8,world
2,2,three,9,10,11.0,12,foo


In [30]:
import sys

In [31]:
data.to_csv(sys.stdout, sep='|')
# 使用了其他的分隔符，这里直接写出到sys.stdout，所以仅仅是打印出来而已

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


    缺失值在输出结果中会被表示为 空字符串，也可以将其表示为别的标记值：

In [32]:
data.to_csv(sys.stdout, na_rep='NULL')
# 用 na_rep 参数来设置填补缺失值

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


    禁止显示行和列的标签

In [35]:
data.to_csv(sys.stdout, index=False, header=False)
# # 列名和索引都被隐藏了
# data.to_csv(sys.stdout, index=False)
# # 隐藏了索引，列名照样显示
# data.to_csv(sys.stdout, header=False)
# # 隐藏了列名，索引依旧会显示

0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


    可以只显示一部分的列，并且按照指定的顺序排列

In [38]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'], na_rep='NULL')
# 原来的列有 something a b c message
# 加了一个填充，NULL

a,b,c
1,2,3.0
5,6,NULL
9,10,11.0


    Series的to_csv方法：

In [39]:
dates = pd.date_range('1/1/2000', periods=7)

# 语法：pandas.date_range(start=None, end=None, periods=None, freq='D', tz=None, normalize=False, name=None, closed=None, **kwargs)

# 该函数主要用于生成一个固定频率的时间索引，在调用构造方法时，必须指定start、end、periods中的两个参数值，否则报错。

# 主要参数说明：

# periods：固定时期，取值为整数或None

# freq：日期偏移量，取值为string或DateOffset，默认为'D'

# normalize：若参数为True表示将start、end参数值正则化到午夜时间戳

# name：生成时间索引对象的名称，取值为string或None

# closed：可以理解成在closed=None情况下返回的结果中，若closed=‘left’表示在返回的结果基础上，再取左开右闭的结果，
# 若closed='right'表示在返回的结果基础上，再取做闭右开的结果


In [40]:
ts = pd.Series(np.arange(7), index=dates)

In [41]:
ts.to_csv('examples/teseries.csv')

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [43]:
pd.read_csv('examples/teseries.csv')

,2000-01-01,0
0,2000-01-02,1
1,2000-01-03,2
2,2000-01-04,3
3,2000-01-05,4
4,2000-01-06,5
5,2000-01-07,6


##### 6.1.3 处理分隔符格式
    大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载，但有时还是需要进行一下手工处理

    对于任何单字符分隔符文件，可以直接使用python内置的csv模块，将任意已打开的文件或文件型对象传给csv.reader：

In [4]:
import csv
f = open('examples/ex7.csv')
reader = csv.reader(f)

In [5]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


    数据整理

In [16]:
# 首先，读取文件到一个多行的列表中：
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [17]:
# 将这些行分为标题行和数据行：
headers, values = lines[0], lines[1:]

In [18]:
lines[0]

['a', 'b', 'c']

In [19]:
lines[1:]

[['1', '2', '3'], ['1', '2', '3']]

In [20]:
# 用字典构造式和zip(*values),后者将行转置为列，创建数据列的字典
data_dict = { h: v for h, v in zip(headers, zip(*values))}

In [21]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

    csv文件的形式有很多。只需定义csv.Dialect的一个子类即可定义出新格式（如专门的分隔符、字符串应用约定、行结束符等）

In [5]:
class my_dialect(csv.Dialect):
    # 用于写操作符的行结束符，默认为" \r\n"，读操作忽略此选项。
    lineterminator = '\n'
    # 用于分隔字段的单字符字符串。默认为","
    delimiter = ';'
    # 用于带有特殊字符（如分隔符）的字段的应用符号。默认为" " "
    quotechar = '"'
    # 引用约定。可选值包括csv.QUOTE（不引用）。默认为QUOTE_MINIMAL
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(data_dict, dialect=my_dialect)

In [27]:
reader

In [28]:
reader = csv.reader(data_dict, delimiter='|')

In [29]:
reader

In [7]:
# 手工输出分隔符文件，可以使用csv.writer。
import csv
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [12]:
pd.read_csv('mydata.csv')
# 使用了自定义的dialect
# 定义了分隔符为 ；
# 定义了结束符为换行

,one;two;three
0,1;2;3
1,4;5;6
2,7;8;9


##### 6.1.4 json数据
    json(JavaScript Object Notation)，是通过http请求在web浏览器和其它应用程序之间发送数据的标准格式之一。是一种比表格型文本格式（如CSV）灵活得多的数据格式。

In [13]:
obj = """
    {
        "name": "Wes",
        "places_lived": ["United States", "Spain", "Germany"],
        "pet": null,
        "siblings": [
            {
                "name": "Scott",
                "age": 30,
                "pets": ["Zeus", "Zuko"]
            },
            {
                "name": "Katie",
                "age": 38,
                "pets": ["Sixes", "Stache", "Cisco"]
            }
        ]
    }
"""
# 处其空值和一些其它的细微差别（如列表末尾不允许存在多余的逗号）之外，JSON
# 非常接近于有效的python代码

In [18]:
obj

'\n    {\n        "name": "Wes",\n        "places_lived": ["United States", "Spain", "Germany"],\n        "pet": null,\n        "siblings": [\n            {\n                "name": "Scott",\n                "age": 30,\n                "pets": ["Zeus", "Zuko"]\n            },\n            {\n                "name": "Katie",\n                "age": 38,\n                "pets": ["Sixes", "Stache", "Cisco"]\n            }\n        ]\n    }\n'

In [15]:
import json

In [16]:
result = json.loads(obj)
# loads 函数，将json 字符串转换为python形式

In [17]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [19]:
# dumps 函数，将python对象转换为json格式
asjson = json.dumps(result)

In [20]:
asjson
# 又变回了字符串

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

    json转换为DataFrame：最简单的是，向DataFrame构造器传入一个字典的列表（就是原先的JSON对象），并选取数据字段的子集

In [21]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])

In [22]:
siblings

,name,age
0,Scott,30
1,Katie,38


    pandas.read_json可以自动将特别格式的JSON数据集转换为Series或DataFrame。

In [24]:
data = pd.read_json('examples/example.json')

In [25]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


    将数据从pandas输出到JSON，使用to_json方法:

In [26]:
print(data.to_json())
#  默认样式为  {index -> {column -> value}}

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [27]:
print(data.to_json(orient='records'))   #  按照 columns----value来还原

# 如果是Series转json，默认的orient是'index'，orient可选参数有 {‘split','records','index'}

# 如果是DataFrame转json，默认的orient是'columns'，orient可选参数有 {‘split','records','index','columns','values'}


# split，样式为 {index -> [index], columns -> [columns], data -> [values]}

# records，样式为[{column -> value}, … , {column -> value}]

# index ，样式为 {index -> {column -> value}}

# columns，样式为 {index -> {column -> value}}

# values，数组样式

# table，样式为{‘schema': {schema}, ‘data': {data}}，和records类似

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


##### 6.1.5 XML和HTML：Web信息收集

    pandas有一个内置的功能，read_html，它可以使用lxml和Beautiful Soup自动将HTML文件中的表格解析为DataFrame对象。

In [28]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')

In [29]:
len(tables)

1

In [30]:
failures = tables[0]

In [32]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [35]:
# 按年份计算倒闭的银行数
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [36]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

##### 6.1.6 利用 lxml.objectify 解析XML

In [37]:
from lxml import objectify

In [38]:
path = 'datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

    root.INDICATOR返回一个用于产生各个<INDICATOR>xml元素的生成器。（是文件中的标签名，getroot返回了节点的引用）

In [40]:
data = []
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [41]:
# 将这个字典转换为DataFrame
perf = pd.DataFrame(data)

In [42]:
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [44]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [45]:
# 访问标签或链接文本中的任何字段（如href）
root

<Element a at 0x1d73c933c08>

In [46]:
root.get('href')

'http://www.google.com'

In [47]:
root.text

'Google'

### 6.2 二进制数据格式

    实现数据的高效二进制格式存储最简单的办法之一就是使用python内置的pickle序列化。pandas对象都有一个用于将数据以pickle格式保存到磁盘上的to_pickle方法：

In [48]:
frame = pd.read_csv('examples/ex1.csv')

In [49]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [50]:
frame.to_pickle('examples/frame_pickle')
# pickle 化数据

In [51]:
# 通过pickle直接读取被pickle化的数据，或是使用更为方便的pandas.read_pickle:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    pickle仅建议用于短期存储格式，因为很难保证该格式永远是稳定的。
    pandas内置支持两个二进制数据格式：HDF5和MessagePack。
    pandas或numpy数据的其他存储格式有：
        bcolz: 一种可压缩的列存储二进制格式，基于Blosc压缩库
        Feather: Feather使用了Apache Arrow的列式内存格式

##### 6.2.1 使用HDF5格式

    HDF5是一种存储大规模科学数据的非常好的文件格式，它可以作为C库，带有很多语言的接口，如java,python,matlab等。
    HDF5中的HDF指的是层次型数据结构。
    能够存储多个数据集并支持元数据。
    对于那些非常大的无法直接放入内存的数据集，HDF5就是很好的选择，因为它可以高效分块读写

In [2]:
frame = pd.DataFrame({'a': np.random.randn(100)})

In [4]:
store = pd.HDFStore('mydata.h5')

In [5]:
store['obj1'] = frame

In [6]:
store['obj1_col'] = frame['a']

In [7]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

    HDF5中的对象可以通过与字典一样的API进行获取

In [10]:
store['obj1']

,a
0,-0.702529
1,-1.432481
2,0.415745
3,-0.473685
4,-1.416298
5,0.381576
6,0.930252
7,0.102764
8,-1.291085
9,-0.376977


    HDF5支持两种存储模式，'fiexd'和'table'。后者通常会更慢，但是支持使用特殊语法进行查询操作：

In [12]:
store.put('obj2', frame, format='table')

In [13]:
store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,-1.223499
11,-0.097365
12,-1.117819
13,-0.627450
14,-1.743127
15,0.341335


In [14]:
store.close()

    put是store['obj2'] = frame方法的显示版本，允许设置其它的选项，比如格式。
    pandas.read_hdf函数可以快捷使用这些工具

In [15]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
# 存储为hdf格式文件，文件名为 mydata.h5 

In [16]:
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])
# 读取刚刚存储的文件，并做了一定的条件查询

,a
0,-0.702529
1,-1.432481
2,0.415745
3,-0.473685
4,-1.416298


##### 6.2.2 读取Microsoft Excel文件

    pandas的ExcelFile类或pandas.read_excel函数支持读取存储在Excel2003及其以上的表格型数据。这两个工具分别使用扩展包xlrd和openpyxl读取XLS和XLSX文件。

In [2]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')

In [5]:
pd.read_excel(xlsx, 'Sheet1')
# pd.read_excel(xlsx)

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


    如果要读取多个表单，创建ExcelFile会更快，但你也可以将文件名传递到pandas.read_excel:

In [6]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')

In [7]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


    将pandas数据写入Excel格式，必须先创建一个ExcelWriter，然后使用pandas对象的to_excel方法将数据写入到其中。

In [8]:
writer = pd.ExcelWriter('examples/ex2.xlsx')

In [9]:
frame.to_excel(writer, 'Sheet1')
#  将ex1文件的数据写入到了ex2文件中
# pandas的 to_xxx 函数，第一个参数都是文件名（文件路径）

In [10]:
writer.save()

In [11]:
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [12]:
pd.read_excel('examples/ex2.xlsx')

,Unnamed: 0,Unnamed: 0.1,a,b,c,d,message
0,0,0,1,2,3,4,hello
1,1,1,5,6,7,8,world
2,2,2,9,10,11,12,foo


In [13]:
#  不使用ExcelWriter，而是传递文件的路径到to_excel。
frame.to_excel('examples/ex2.xlsx')

### 6.3 Web APIs交互

In [14]:
import requests

In [17]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
# github上的pandas主题

In [18]:
resp = requests.get(url)

In [19]:
resp

<Response [200]>

In [20]:
data = resp.json()
# 返回一个json字典，加载到一个python对象中

In [21]:
data[0]['title']

'Enhancement: The shift function should have an optional argument allowing to extend the dataframe i.e. for forecasts'

In [24]:
# data中的每一个元素都是一个包含所有github主题也数据（不包含评论）的字典。
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [25]:
issues

,number,title,labels,state
0,27091,Enhancement: The shift function should have an...,[],open
1,27090,ENH: allow non-Tick offsets in index.round/cei...,[],open
2,27089,Anyone knows a free Transaction Language 1 (TL...,[],open
3,27088,BUG: Fix handling of ambiguous or nonexistent ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,27087,"WIP: Add DayEnd, DayBegin Offsets (Help Wanted)",[],open
5,27086,TST: Add missing tests for loc slicing of Peri...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
6,27085,TST: Split test_offsets.py,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open
7,27084,DEPR: msgpack,"[{'id': 87485152, 'node_id': 'MDU6TGFiZWw4NzQ4...",open
8,27081,API: .equals for Extension Arrays,"[{'id': 35818298, 'node_id': 'MDU6TGFiZWwzNTgx...",open
9,27080,Bug in SparseArray.__array_ufunc__ for reduce,[],open


### 6.4 数据库交互

In [27]:
# SQLite数据库（通过python内置的sqlite3驱动器）
import sqlite3

In [28]:
query = """
    CREATE TABLE test(
        a VARCHAR(20),
        b VARCHAR(20),
        c REAL,
        d INTEGER
    );
"""

In [29]:
# 连接数据库
con = sqlite3.connect('mydata.sqlite')

In [31]:
# 执行语句（创建表test）
con.execute(query)

In [33]:
#  提交， 成功创建表test
con.commit()
# 三个固有的步骤

In [34]:
# 插入数据
data = [('Atlanta', 'Georgia', 1.25, 6), ('Tallahasee', 'Florida', 2.6, 3), ('Sacramento', 'California', 1.7, 5)]

In [35]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [37]:
con.executemany(stmt, data)

    从表中选取数据时，大部分python sql驱动器都会返回一个元祖列表

In [40]:
cursor = con.execute('select * from test')

In [43]:
cursor

In [41]:
rows = cursor.fetchall()
# fetchall()函数

In [42]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahasee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

    将元祖列表传递给DataFrame构造器，但还需要列名（位于cursor的description属性中）

In [44]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [47]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])
# 索引为整数，列名为  a b c d (上面的cursor.description)
# pd.DataFrame(rows)
#  列名和索引都是整数 0 1 2 3

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahasee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [49]:
# 简化操作
import sqlalchemy as sqla

In [50]:
db = sqla.create_engine('sqlite:///mydata.sqlite')
# 连接数据库

In [52]:
# 从之前创建的表中读取数据
pd.read_sql('select * from test', db)
# read_sql 函数，从SQLAlchemy连接读取数据

,a,b,c,d
